In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib import rnn
mnist = input_data.read_data_sets('/tmp/data',one_hot = True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
batch_size = 128
hidden_num = 128
chunks = 28
chunk_size =28
classes = 10
epoches = 3

In [3]:
data = tf.placeholder(tf.float32, [None,chunks,chunk_size])
target = tf.placeholder(tf.float32, [None,classes])

In [4]:
cell = rnn.LSTMCell(hidden_num,state_is_tuple = True)

In [5]:
value, state = tf.nn.dynamic_rnn(cell,data,dtype = tf.float32)

In [6]:
value = tf.transpose(value,[1,0,2])
#print(value.shape)
last_val = tf.gather(value, int(value.get_shape()[0]) - 1)

In [7]:
output = {'weights':tf.Variable(tf.truncated_normal([hidden_num,classes])),
         'bias':tf.Variable(tf.truncated_normal([classes]))}

In [8]:
prediction = tf.nn.softmax(tf.matmul(last_val,output['weights'])+output['bias'])

In [9]:
cost = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits = prediction,labels = target))

In [10]:
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cost)

/Users/zhaomingchuan/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [11]:
correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(target, 1))
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

In [12]:
init = tf.global_variables_initializer()

In [13]:
sess = tf.Session()
sess.run(init)

In [14]:
for epoch in range(epoches):
    loss = 0
    for _ in range(int(mnist.train.num_examples/batch_size)):
        epoch_x, epoch_y = mnist.train.next_batch(batch_size)
        epoch_x = epoch_x.reshape([batch_size,chunks,chunk_size])
        
        _,c = sess.run([minimize,cost], feed_dict= {data:epoch_x, target:epoch_y})
        loss += c
    accu = accuracy.eval(session = sess,feed_dict = {data: mnist.test.images.reshape((-1,chunks,chunk_size)), target: mnist.test.labels})    
    print('loss:{}'.format(loss))
    print('accuracy:{}'.format(accu))

loss:97306.00624084473
accuracy:0.867900013923645
loss:83992.38243103027
accuracy:0.9456999897956848
loss:82661.67802429199
accuracy:0.9641000032424927
